In [ ]:
import numpy as np
import pandas as pd
import keras
from keras import layers, models
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, Adamax, SGD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from google.colab import files

# Esimerkki
'''

# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

'''

# Ladataan data CSV-tiedostosta
file_path = "data_from_mysql_where_g16.csv"
data = pd.read_csv(file_path)

# Tarkastetaan datan rakenne
print(data.head())

# Suodatetaan pois kaikki rivit, joissa 'sensorvalue_d' on 0
data_filtered = data[data['sensorvalue_d'] != 0]

x_data = data_filtered[['sensorvalue_a', 'sensorvalue_b', 'sensorvalue_c']].values  # x, y, z
y_data = data_filtered['sensorvalue_d'].values # suunta

# Skaalataan syötteet [0, 1] väliin käyttäen StandardScaler
scaler = StandardScaler()
x_data = scaler.fit_transform(x_data.astype('float32'))

# Suuntaa on 6 luokkaa
num_classes = 6
y_data = keras.utils.to_categorical(y_data - 1, num_classes)

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")

# Määritellään malli
model = keras.Sequential(
    [
        keras.Input(shape=(x_train.shape[1],)),  # Syötemuoto (x, y, z)
        layers.Dense(256, activation="relu"),  # Ensimmäinen tiheä kerros
        layers.Dropout(0.2),  # Dropout, jotta ylikoulutus ei tapahdu
        layers.Dense(128, activation="relu"),  # Toinen tiheä kerros
        layers.LeakyReLU(negative_slope=0.2),  # Vaihtoehto ReLU:lle
        layers.Dense(64, activation="relu"),  # Kolmas tiheä kerros
        layers.Dense(32, activation="relu"),
        layers.Dense(num_classes, activation="softmax"),  # Lopullinen luokittelukerros
    ]
)

model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Koulutetaan mallia
batch_size = 128
epochs = 100

optimizer = Adam(learning_rate=0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[early_stopping])

# Arvioidaan malli testidatalla
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

# Tallennetaan malli (painot ja rakenne)
model.save('my_model.keras')

# Osa 2

# Haetaan mallin painot
weights = model.get_weights()

# Painot ovat lista, jossa on numpy-taulukoita
for idx, weight in enumerate(weights):
    print(f"Painojen ja biasin {idx} muoto: {weight.shape}")
    #print(f"Painot ja bias {idx}:", weight)

# Aktivointifunktiot
def relu(x):
    return np.maximum(0, x)  # ReLU-funktio

def softmax(x):
    exp_values = np.exp(x - np.max(x))  # Stabiili softmax
    return exp_values / exp_values.sum(axis=-1, keepdims=True)

# Syöte
x = data_filtered['sensorvalue_a'].values
y = data_filtered['sensorvalue_b'].values
z = data_filtered['sensorvalue_c'].values

# esim.
#x = 1200
#y = 1500
#z = 1800


# Esimerkiksi käytetään vain ensimmäistä riviä syötteeksi:
input_data = np.array([x[0], y[0], z[0]]).reshape(1, -1)  # Muutetaan muotoon (1, 3)

#input_data = np.array([x, y, z]).reshape(1, -1)  # Muutetaan syöte oikeaan muotoon  # X, Y, Z syötteet

# Esimerkki: painot ja biasit manuaalisesti määriteltynä
weights_0 = np.random.randn(3, 256)  # Painot piilokerrokselle 1 (3 syötettä, 256 neuronin koko)
bias_0 = np.random.randn(256)       # Bias piilokerrokselle 1

weights_1 = np.random.randn(256, 128)  # Painot piilokerrokselle 2 (256 -> 128)
bias_1 = np.random.randn(128)         # Bias piilokerrokselle 2

weights_2 = np.random.randn(128, 64)   # Painot piilokerrokselle 3 (128 -> 64)
bias_2 = np.random.randn(64)          # Bias piilokerrokselle 3

weights_3 = np.random.randn(64, 32)    # Painot piilokerrokselle 4 (64 -> 32)
bias_3 = np.random.randn(32)          # Bias piilokerrokselle 4

weights_4 = np.random.randn(32, 6)     # Painot ulostulokerrokselle (32 -> 6)
bias_4 = np.random.randn(6)           # Bias ulostulokerrokselle

# Etenee syötteestä piilokerrosten kautta ulostuloon
def forward_propagation(input_data):
    # Piilokerros 1
    z0 = np.dot(input_data, weights_0) + bias_0
    a0 = relu(z0)  # Aktivointi

    # Piilokerros 2
    z1 = np.dot(a0, weights_1) + bias_1
    a1 = relu(z1)  # Aktivointi

    # Piilokerros 3
    z2 = np.dot(a1, weights_2) + bias_2
    a2 = relu(z2)  # Aktivointi

    # Piilokerros 4
    z3 = np.dot(a2, weights_3) + bias_3
    a3 = relu(z3)  # Aktivointi

    # Ulostulokerros
    z4 = np.dot(a3, weights_4) + bias_4
    output = softmax(z4)  # Softmax aktivointi

    return output

# Laske tulos syötteelle (x, y, z)
result = forward_propagation(input_data)

# Tulosta tulos
print("Verkon ulostulo:", result)

# Tallenna painot ja biasit header-tiedostoon
header_file = "neuroverkonKertoimet.h"

with open(header_file, "w") as f:
    f.write("#ifndef NEUROVERKONKERTOIMET_H\n")
    f.write("#define NEUROVERKONKERTOIMET_H\n\n")

    # Kirjoita painot ja biasit jokaiselle kerrokselle
    for idx, weight in enumerate(weights):
        if len(weight.shape) == 2:  # Painot (matriisi)
            f.write(f"float weights_{idx}[{weight.shape[0]}][{weight.shape[1]}] = {{\n")
            for row in weight:
                f.write("    {" + ", ".join(map(str, row)) + "},\n")
            f.write("};\n\n")
        elif len(weight.shape) == 1:  # Bias (vektori)
            f.write(f"float biases_{idx}[{weight.shape[0]}] = {{")
            f.write(", ".join(map(str, weight)))
            f.write("};\n\n")

    f.write("#endif // NEUROVERKONKERTOIMET_H\n")



   id            timestamp  groupid  from_mac  to_mac  sensorvalue_a  \
0  39  2024-11-14 12:30:54       16         0       0         1510.0   
1  40  2024-11-14 12:30:54       16         0       0         1503.0   
2  41  2024-11-14 12:30:54       16         0       0         1511.0   
3  42  2024-11-14 12:30:55       16         0       0         1509.0   
4  43  2024-11-14 12:30:56       16         0       0         1505.0   

   sensorvalue_b  sensorvalue_c  sensorvalue_d  sensorvalue_e  sensorvalue_f  
0         1405.0         1812.0            0.0            0.0              0  
1         1413.0         1818.0            0.0            0.0              0  
2         1416.0         1814.0            0.0            0.0              0  
3         1412.0         1814.0            0.0            0.0              0  
4         1410.0         1804.0            0.0            0.0              0  
x_train shape: (2210, 3)
x_test shape: (553, 3)


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_60 (Dense)                     │ (None, 256)                 │           1,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_61 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_12 (LeakyReLU)           │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_62 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_63 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_64 (Dense)                     │ (None, 6)                   │             198 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 44,454 (173.65 KB)

 Trainable params: 44,454 (173.65 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 152ms/step - accuracy: 0.4211 - loss: 1.6923 - val_accuracy: 0.6063 - val_loss: 1.2619
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6210 - loss: 1.1517 - val_accuracy: 0.6154 - val_loss: 0.8753
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6262 - loss: 0.8981 - val_accuracy: 0.6923 - val_loss: 0.8158
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6533 - loss: 0.8071 - val_accuracy: 0.6606 - val_loss: 0.7979
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6689 - loss: 0.7691 - val_accuracy: 0.6516 - val_loss: 0.7790
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6859 - loss: 0.8130 - val_accuracy: 0.6742 - val_loss: 0.7758
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6357 - loss: 0.8133 - val_accuracy: 0.6923 - val_loss: 0.7654
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6930 - loss: 0.7765 - val_accuracy: 0.6652 -